In [1]:
# ch5/porto_seguro’s_safe_driver_prediction_xgb_modeling.ipynb

import numpy as np
import pandas as pd

# 데이터 경로
data_path = '/kaggle/input/porto-seguro-safe-driver-prediction/'

train = pd.read_csv(data_path + 'train.csv', index_col='id')
test = pd.read_csv(data_path + 'test.csv', index_col='id')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col='id')

In [2]:
all_data = pd.concat([train, test], ignore_index=True)
all_data = all_data.drop('target', axis=1) # 타깃 값 제거

all_features = all_data.columns.tolist() # 전체 피처

### 결측값 개수를 파생 변수로 생성

In [3]:
# '데이터 하나당 결측값 개수'를 파생 변수로 추가
all_data['num_missing'] = (all_data==-1).sum(axis=1)

# 명목형 피처, 태그에 calc가 달린 피처를 제외한 피처
remaining_features = [col for col in all_features \
                      if ('cat' not in col and 'calc' not in col)] 
# num_missing을 remaining_features에 추가
remaining_features.append('num_missing')

### 명목형 피처 원-핫 인코딩 적용

In [4]:
from sklearn.preprocessing import OneHotEncoder

cat_features = [col for col in all_features if 'cat' in col] # 명목형 피처

# 원-핫 인코딩 적용
onehot_encoder = OneHotEncoder()
encoded_cat_matrix = onehot_encoder.fit_transform(all_data[cat_features]) 

### `ind` 변수의 고유값을 조합한 파생 변수 `mix_ind` 생성

In [5]:
# 태그에 ind가 달린 피처
ind_features = [col for col in all_features if 'ind' in col]

first_col=True
for col in ind_features:
    if first_col:
        all_data['mix_ind'] = all_data[col].astype(str)+'_'
        first_col = False
    else:
        all_data['mix_ind'] += all_data[col].astype(str)+'_'

In [6]:
cat_count_features = []
for col in cat_features+['mix_ind']:
    val_counts_dic = all_data[col].value_counts().to_dict()
    all_data[f'{col}_count'] = all_data[col].apply(lambda x: val_counts_dic[x])
    cat_count_features.append(f'{col}_count')

In [7]:
from scipy import sparse

drop_features = ['ps_ind_14', 'ps_ind_10_bin','ps_ind_11_bin', 
                 'ps_ind_12_bin','ps_ind_13_bin','ps_car_14']

# remaining_features, cat_count_features에서 drop_features를 제거한 데이터
all_data_remaining = all_data[remaining_features+cat_count_features].drop(drop_features, axis=1)

# 데이터 합치기
all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data_remaining),
                               encoded_cat_matrix],
                              format='csr')

In [8]:
num_train = train.shape[0] # 훈련 데이터 개수

# 훈련 데이터와 테스트 데이터 나누기
X = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train:]

y = train['target'].values

In [9]:
def eval_gini(y_true, y_pred):
    # 실제 값과 예측 값의 크기가 같은지 확인 (값이 다르면 오류 발생)
    assert y_true.shape == y_pred.shape

    n_samples = y_true.shape[0] # 데이터 개수
    L_mid = np.linspace(1 / n_samples, 1, n_samples) # 대각선 값

    # 1) 예측 값에 대한 지니계수
    pred_order = y_true[y_pred.argsort()] # y_pred 크기순으로 y_true 값 정렬
    L_pred = np.cumsum(pred_order) / np.sum(pred_order) # 로렌츠 곡선
    G_pred = np.sum(L_mid - L_pred)# 예측 값에 대한 지니계수

    # 2) 예측이 완벽할 때 지니계수
    true_order = y_true[y_true.argsort()] # y_true 크기순으로 y_true 값 정렬
    L_true = np.cumsum(true_order) / np.sum(true_order) # 로렌츠 곡선
    G_true = np.sum(L_mid - L_true) # 예측이 완벽할 때 지니계수

    # 정규화된 지니계수
    return G_pred / G_true

In [10]:
def gini(preds, dtrain):
    labels = dtrain.get_label()
    return 'gini', eval_gini(labels, preds)

## BayesianOptimization을 활용한 하이퍼 파라미터 최적화

In [11]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

# 8:2 비율로 훈련 데이터, 검증 데이터 분리 (베이지안 최적화 수행용)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size=0.2, 
                                                      random_state=0)
# 베이지안 최적화용 데이터 세트
bayes_dtrain = xgb.DMatrix(X_train, y_train)
bayes_dvalid = xgb.DMatrix(X_valid, y_valid)

In [12]:
# 베이지안 최적화를 위한 하이퍼 파라미터 범위
param_bounds = {'max_depth': (4, 8),
                'subsample': (0.7, 0.9),
                'colsample_bytree': (0.7, 0.9),
                'min_child_weight': (5, 7),
                'gamma': (9, 11),
                'reg_alpha': (7, 9),
                'reg_lambda': (1.2, 1.4),
                'scale_pos_weight': (1.5, 1.7)}

# 고정된 하이퍼 파라미터
fixed_params = {'objective': 'binary:logistic',
                'learning_rate': 0.05,
                'random_state': 1991}

In [13]:
def eval_function(max_depth, subsample, colsample_bytree, min_child_weight,
                 reg_alpha, gamma, reg_lambda, scale_pos_weight):
    '''최적화하려는 평가지표(지니계수) 계산 함수'''
    
    params = {'max_depth': int(round(max_depth)),
              'subsample': subsample,
              'colsample_bytree': colsample_bytree,
              'min_child_weight': int(round(min_child_weight)),
              'gamma': gamma,
              'reg_alpha':reg_alpha,
              'reg_lambda': reg_lambda,
              'scale_pos_weight': scale_pos_weight}
    # 고정된 하이퍼 파라미터도 추가
    params.update(fixed_params)
    
    print('하이퍼 파라미터:', params)    
        
    # XGBoost 모델 훈련
    xgb_model = xgb.train(params=params, 
                          dtrain=bayes_dtrain,
                          num_boost_round=500,
                          evals=[(bayes_dvalid, 'bayes_dvalid')],
                          maximize=True,
                          feval=gini,
                          early_stopping_rounds=150,
                          verbose_eval=False)
    
                           
    best_iter = xgb_model.best_iteration # 최적 반복 횟수
    # 검증 데이터로 예측 수행
    preds = xgb_model.predict(bayes_dvalid, ntree_limit=best_iter)
    # 지니계수 계산
    gini_score = eval_gini(y_valid, preds)
    print(f'지니계수: {gini_score}\n')
    
    return gini_score

In [14]:
from bayes_opt import BayesianOptimization

# 베이지안 최적화 객체 생성
optimizer = BayesianOptimization(f=eval_function, 
                                 pbounds=param_bounds,
                                 random_state=0)

In [15]:
# 베이지안 최적화 수행
optimizer.maximize(init_points=2, n_iter=2)

|   iter    |  target   | colsam... |   gamma   | max_depth | min_ch... | reg_alpha | reg_la... | scale_... | subsample |
-------------------------------------------------------------------------------------------------------------------------
하이퍼 파라미터: {'max_depth': 6, 'subsample': 0.878354600156416, 'colsample_bytree': 0.809762700785465, 'min_child_weight': 6, 'gamma': 10.430378732744838, 'reg_alpha': 7.84730959867781, 'reg_lambda': 1.3291788226133312, 'scale_pos_weight': 1.5875174422525384, 'objective': 'binary:logistic', 'learning_rate': 0.05, 'random_state': 1991}
[05:25:34] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
지니계수: 0.2827732646026083

|  1        |  0.2828   |  0.8098   |  10.43    |  6.411    |  6.09     |  7.847    |  1.329    |  1.588    |  0.8784   |
하이퍼 파라미터: {'max_depth': 

In [16]:
# 평가함수 점수가 최대일 때 하이퍼 파라미터
max_params = optimizer.max['params']
max_params

{'colsample_bytree': 0.8927325521002059,
 'gamma': 9.766883037651555,
 'max_depth': 7.166900152330658,
 'min_child_weight': 6.0577898395058085,
 'reg_alpha': 8.136089122187865,
 'reg_lambda': 1.385119327658532,
 'scale_pos_weight': 1.5142072116395773,
 'subsample': 0.717425859940308}

In [17]:
# 정수형 하이퍼 파라미터 변환
max_params['max_depth'] = int(round(max_params['max_depth']))

In [18]:
max_params

{'colsample_bytree': 0.8927325521002059,
 'gamma': 9.766883037651555,
 'max_depth': 7,
 'min_child_weight': 6.0577898395058085,
 'reg_alpha': 8.136089122187865,
 'reg_lambda': 1.385119327658532,
 'scale_pos_weight': 1.5142072116395773,
 'subsample': 0.717425859940308}

In [19]:
# 고정된 하이퍼 파라미터 추가
max_params.update(fixed_params)

In [20]:
max_params

{'colsample_bytree': 0.8927325521002059,
 'gamma': 9.766883037651555,
 'max_depth': 7,
 'min_child_weight': 6.0577898395058085,
 'reg_alpha': 8.136089122187865,
 'reg_lambda': 1.385119327658532,
 'scale_pos_weight': 1.5142072116395773,
 'subsample': 0.717425859940308,
 'objective': 'binary:logistic',
 'learning_rate': 0.05,
 'random_state': 1991}

In [21]:
from sklearn.model_selection import StratifiedKFold

# Stratified K 폴드 교차검증기 생성
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)

oof_preds = np.zeros(X.shape[0]) # OOF 예측용 1차원 배열
test_preds = np.zeros(X_test.shape[0]) # 테스트 데이터 예측용 1차원 배열

# OOF 방식으로 모델 훈련, 검증, 예측
for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    # 각 폴드를 구분하는 문구
    print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}', '#'*40)
    
    # 훈련용 데이터, 검증용 데이터 설정
    X_train, y_train = X[train_idx], y[train_idx]
    X_valid, y_valid = X[valid_idx], y[valid_idx]

    # xgb 데이터세트 생성 
    dtrain = xgb.DMatrix(X_train, y_train)
    dvalid = xgb.DMatrix(X_valid, y_valid)
    dtest = xgb.DMatrix(X_test)

    xgb_model = xgb.train(params=max_params, 
                          dtrain=dtrain,
                          num_boost_round=1000,
                          evals=[(dvalid, 'valid')],
                          maximize=True,
                          feval=gini,
                          early_stopping_rounds=150,
                          verbose_eval=100)

    # 모델 성능이 가장 좋을 때의 부스팅 반복 횟수 저장
    best_iter = xgb_model.best_iteration
    # 테스트 데이터를 활용해 예측
    test_preds += xgb_model.predict(dtest, 
                                    ntree_limit=best_iter)/folds.n_splits
    
    # 모델 성능 평가를 위한 oof 예측 
    oof_preds[valid_idx] += xgb_model.predict(dvalid, ntree_limit=best_iter)

    # oof 예측확률에 대한 정규화 지니계수
    gini_score = eval_gini(y_valid, oof_preds[valid_idx])
    print(f'폴드 {idx+1} 지니계수: {gini_score}\n')

######################################## 폴드 1 / 폴드 5 ########################################
[05:46:27] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	valid-logloss:0.65282	valid-gini:0.15941
[100]	valid-logloss:0.15637	valid-gini:0.28555
[200]	valid-logloss:0.15485	valid-gini:0.29589
[300]	valid-logloss:0.15475	valid-gini:0.29885
[400]	valid-logloss:0.15474	valid-gini:0.29912
[500]	valid-logloss:0.15473	valid-gini:0.29909
[600]	valid-logloss:0.15473	valid-gini:0.29956
[700]	valid-logloss:0.15475	valid-gini:0.29816
[757]	valid-logloss:0.15482	valid-gini:0.29692
폴드 1 지니계수: 0.29961440115120136

######################################## 폴드 2 / 폴드 5 ########################################
[05:55:37] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used w

In [22]:
print('OOF 지니계수:', eval_gini(y, oof_preds))

OOF 지니계수: 0.28849001909297844


In [23]:
submission['target'] = test_preds
submission.to_csv('submission.csv')

- 4: 0.287283075088194
- 5: 0.2879975214639839 / num_boost_round = 400 -> 1000으로 늘림 (0.29085)
- 6: 0.28849001909297844 / learning_rate': 0.03 -> 0.05로 수정
- 7: 재생산 코드
- 8: 0.2882575420502362 한글화 작업 / # 베이지안 최적화 수행 반복횟수 5로 올림 (0.29078)
- 9: 0.28878160543816095 / optimizer.maximize(init_points=3, n_iter=7) (0.29115)
- 10: (init_points=2, n_iter=2)
- 12:  OOF 지니계수: 0.28849001909297844 / evals=[(dvalid, 'valid')] 사용 마지막 폴드 지니계수 일치여부 확인
- 13: 최종 코드

In [24]:
max_params

{'colsample_bytree': 0.8927325521002059,
 'gamma': 9.766883037651555,
 'max_depth': 7,
 'min_child_weight': 6.0577898395058085,
 'reg_alpha': 8.136089122187865,
 'reg_lambda': 1.385119327658532,
 'scale_pos_weight': 1.5142072116395773,
 'subsample': 0.717425859940308,
 'objective': 'binary:logistic',
 'learning_rate': 0.05,
 'random_state': 1991}